In [ ]:
#trying this tutorial: https://www.natekratzer.com/posts/census_map/
import numpy as np
import geopandas as gpd
import requests

In [ ]:
#reading in file and mapping counties using WI as example
county_shp = gpd.read_file('cb_2022_us_county_500k.zip')
#wi_counties = county_shp[county_shp.STATE_NAME == 'Wisconsin']

In [ ]:
#wi_counties.explore()

In [ ]:
#load API key
with open('census_api_key.txt') as key:
    api_key=key.read().strip()

#specify the data source by year and survey
year = '2020'
dsource = 'dec' #Decennial Census
dname = 'dhc' #Demographic and Housing Characteristics
base_url = f'https://api.census.gov/data/{year}/{dsource}/{dname}'

#unique to this specific data request
cols = 'NAME,P1_001N' #NAME of geography as well as the variables I want to pull
geo = 'county:*' #county geography level
state = 'state:*' #all states

#add unique request features to the base_url
data_url = f'{base_url}?get={cols}&for={geo}&in={state}&key={api_key}'

#go get the data
response = requests.get(data_url)



In [ ]:
#take the data in json and format it into a dataframe
data = response.json()

In [ ]:
df = pd.DataFrame(data = data[1:], columns = data[0])
df.head()

In [ ]:
#rename columns and convert population to numeric
df.rename(columns = {'NAME': 'county_name', 'P1_001N': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
df['population'] = df['population'].apply(pd.to_numeric)

In [ ]:
df['fips'] = df['state_fips'] + df['county_fips']
df.info()

In [ ]:
states = county_shp.STATE_NAME.values.tolist()
print(states)

In [ ]:
#creating a full US map
usa_counties = county_shp[county_shp.STATE_NAME == states]


In [ ]:
usa_counties.explore()

In [ ]:
#construct the fips code to match with the geography data
usa_counties['fips'] = usa_counties['STATEFP'] + usa_counties['COUNTYFP']
usa_counties.head()

In [ ]:
#merge in the new data
#clean up the names to be a bit more presentable
df_map = (usa_counties.merge(df, how = 'left', on = 'fips').rename(columns = {'NAMELSAD': 'County Name', 'population': 'County Population'}))

df_map.head()

In [ ]:
df_map.explore(
    column ='County Population',
    scheme="naturalbreaks",  # use mapclassify's natural breaks scheme
    legend=True,  # show legend
    k=5,  # use 5 bins
    tooltip = ['County Name', 'County Population'],
    tiles = 'CartoDB positron', #fades into the background better than the default
    legend_kwds=dict(colorbar=False, color="gray")
)

In [ ]:
biggest_counties = df_map[df_map['County Population'] > 5000000]
biggest_counties

In [ ]:
#another approach I found, but did not work well bc of size of dataset and rate limits
#pip install geopy

In [ ]:
#note: rate limit became a problem; will need to filter down to CA data only before retrying
#from geopy.geocoders import Nominatim

#def get_zip_code(latitude, longitude): 
    #geolocator = Nominatim(user_agent="mafphd@icloud.com")
    #location = geolocator.reverse((latitude, longitude), exactly_one=True)
    #address = location.raw['address']
    #zip_code = address.get('postcode')
    
    #return zip_code

In [ ]:
import pandas as pd
burn = pd.read_csv('Burn_Severity_Trends.csv')
burn.info()

In [ ]:
#burn['ZIP_CODE'] = burn.apply(lambda row: get_zip_code(row['LATITUDE'], row['LONGITUDE']), axis=1)

In [ ]:
#trying to identify counties by latitude/longitude coordinates using TIGER/Line Shapefile, 2019, nation, U.S., Current County and Equivalent National Shapefile from data.gov
import geopandas as gpd

# Load the shapefile
gdf = gpd.read_file('tl_2019_us_county/tl_2019_us_county.shp') #file is too large to commit; download from https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-county-and-equivalent-national-shapefile

def get_county_name(latitude, longitude):
    point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([longitude], [latitude]))
    county = gdf[gdf.contains(point.unary_union)]
    return county['NAME'].iloc[0] if not county.empty else None

# Example usage
latitude = 37.7749  # Example latitude
longitude = -122.4194  # Example longitude

county_name = get_county_name(latitude, longitude)
print("County Name:", county_name)


In [ ]:
burn['County'] = burn.apply(lambda row: get_county_name(row['LATITUDE'], row['LONGITUDE']), axis=1)

In [ ]:
burn['County'].isna().sum()

In [ ]:
#looking at the missing counties to see if any are within approximate CA coordinates (from ChatGPT/verified on a map: min_lat = 32.5, max_lat = 37.0,min_lon = -125.0,max_lon = -114.0)
missing_ctys = burn[burn['County'].isna()]
missing_ctys_sorted = missing_ctys[['LATITUDE', 'LONGITUDE']]
missing_ctys_sorted.sort_values(by='LATITUDE', ascending = True)

In [ ]:
#using BallTree nearest neighbors approach to fill in County based on closest known lat/long coordinates
from sklearn.neighbors import BallTree

# Assuming 'burn' is your original DataFrame

# Create a subset DataFrame with only rows that have County information
known_counties = burn.dropna(subset=['County'])

# Extract latitude and longitude for known counties
known_coordinates = known_counties[['LATITUDE', 'LONGITUDE']].values

# Build a BallTree for efficient nearest neighbor search
tree = BallTree(known_coordinates, leaf_size=15)

# Define a function to impute missing County values based on nearest neighbors
def impute_county(row):
    if pd.isna(row['County']):
        distances, indices = tree.query([[row['LATITUDE'], row['LONGITUDE']]], k=1)
        closest_county = known_counties.iloc[indices[0][0]]['County']
        return closest_county
    else:
        return row['County']

# Apply the impute_county function to fill in missing values
burn['County'] = burn.apply(impute_county, axis=1)


In [ ]:
burn['County'].isna().sum()

In [ ]:
#checking to see if I can get fips codes from the shapefile to join the two dataframes
gdf.info()

In [ ]:
# bringing FP codes to the burn dataframe for joining
# Spatial join between burn DataFrame and shapefile
gdf = gdf[['STATEFP', 'COUNTYFP', 'geometry']]  # Select only necessary columns
burn_gdf = gpd.GeoDataFrame(burn, geometry=gpd.points_from_xy(burn.LONGITUDE, burn.LATITUDE), crs=gdf.crs)
merged = gpd.sjoin(burn_gdf, gdf, how='left', op='within')

# Extract STATEFP and COUNTYFP values
burn['state_fips'] = merged['STATEFP']
burn['county_fips'] = merged['COUNTYFP']

# Create 'fips' column by concatenating 'state_fips' and 'county_fips'
burn['fips'] = burn['state_fips'].astype(str) + burn['county_fips'].astype(str)


In [ ]:
burn.head()

In [ ]:
# filtering down to California only for both dataframes
Cali = df_map[df_map['STATE_NAME']=='California']
Cali.head()

In [ ]:
Cali_burn = burn[burn['state_fips'] == '06']
Cali_burn.info()

In [ ]:
burn_pop_merge = Cali.merge(Cali_burn, how = 'left', on = 'fips')
burn_pop_merge.info()

In [ ]:
#dropping extra columns
burn_pop_merge.drop(['county_name', 'state_fips_y', 'county_fips_y', 'County'], axis=1, inplace=True)
burn_pop_merge.info()

In [ ]:
extras = burn_pop_merge[burn_pop_merge['POST_ID'].isna()]
print(extras)


In [ ]:
extras.info()

In [ ]:
#dropping the two extra rows that got added, since they don't seem to have any info from the burn data
burn_pop_merge = burn_pop_merge.dropna(subset=['POST_ID'])
burn_pop_merge.info()

In [ ]:
pip install seaborn

In [ ]:
# quick visualization of the results
import matplotlib.pyplot as plt
import seaborn as sns
# Group by 'County Name' and get the count of POST_ID
county_counts = burn_pop_merge.groupby('County Name')['POST_ID'].count()

# Filter for counties with more than 5 POST_ID values
county_counts_filtered = county_counts[county_counts > 5]

# Merge with the original DataFrame to keep 'County Population'
merged_df = burn_pop_merge.merge(county_counts_filtered, left_on='County Name', right_index=True, suffixes=('_original', '_count'))

# Sort counties by County Population in descending order
sorted_df = merged_df.sort_values(by='County Population', ascending=False)
# Create the bar chart
plt.figure(figsize=(20, 6))
sns.barplot(x=sorted_df['County Name'], y=sorted_df['POST_ID_count'],color='firebrick')
plt.title('Counties with > 5 fire records, Sorted by Population')
plt.xlabel('County Name')
plt.ylabel('Fires')
plt.xticks(rotation=90)
plt.show()
